In [ ]:
# Dependencies and setup
import numpy as np
import os
import pandas as pd

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc, Column, Integer, String, Float
from config import (user, password, host, port, database)
# from TAconfig import (user, password, host, port, database)

# Extract

### A. &nbsp;National Parks (from JSON)
- National Parks with correct lat/lng coordinates.
- Incomplete dataset.

In [ ]:
# Data source URL and variable
aURL = "https://github.com/learn-chef/national-parks-java/blob/master/national-parks.json"
aFilePath = "data/natParks.json"
print(f"{aFilePath} is {round(os.path.getsize(aFilePath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{aURL}")

In [ ]:
# Read JSON file into DataFrame
a_df = pd.read_json(aFilePath)
a_df.head()

### B. &nbsp;National Parks (from CSV)
- All National Parks.
- Complete dataset of National Park names, but missing 102 lat/lng coordinates.

In [ ]:
# Data source URL and variable
bURL = "https://public-nps.opendata.arcgis.com/datasets/nps-boundary-1/data?geometry=79.963%2C-20.479%2C-104.959%2C70.899"
bFilePath = "data/natParks.csv"
print(f"{bFilePath} is {round(os.path.getsize(bFilePath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{bURL}")

In [ ]:
# Read CSV file into DataFrame
bRaw_df = pd.read_csv(bFilePath)
bRaw_df.head()

### C. &nbsp;National Parks with Size/Area (from CSV)
- All National Parks; complete dataset.

In [ ]:
# Data source URL and variable
cURL = "https://public-nps.opendata.arcgis.com/datasets/nps-boundary-1/data?geometry=79.963%2C-20.479%2C-104.959%2C70.899"
cFilePath = "data/natParksArea.csv"
print(f"{cFilePath} is {round(os.path.getsize(cFilePath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{cURL}")

In [ ]:
# Read CSV file into DataFrame
cRaw_df = pd.read_csv(cFilePath)
cRaw_df.head()

### Points of Interest
- All Points of Interest within each National Park with lat/lng coordinates.
- Complete dataset.

In [ ]:
# Data source URL and variable
rawPointsURL = "https://public-nps.opendata.arcgis.com/datasets/nps-points-of-interest-pois-geographic-coordinate-system-1?geometry=79.973%2C-21.130%2C-104.949%2C70.669"
rawPointsFilePath = "data/pointsInterest.csv"
print(f"{rawPointsFilePath} is {round(os.path.getsize(rawPointsFilePath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{rawPointsURL}")

In [ ]:
# Read CSV file into DataFrame
rawPoints_df = pd.read_csv(rawPointsFilePath, low_memory = False)
rawPoints_df.head()

### Attendance/Visitors (Recreation Visits)
- Dataset does not include attendance numbers for trails and other unstaffed National Park locations.

In [ ]:
# Data source URL and variable
attendanceURL = "https://irma.nps.gov/STATS/SSRSReports/National%20Reports/Annual%20Visitation%20By%20Park%20(1979%20-%20Last%20Calendar%20Year)"
attendanceFilePath = "data/finalAttendance.csv"
print(f"{attendanceFilePath} is {round(os.path.getsize(attendanceFilePath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{attendanceURL}")

In [ ]:
# Read CSV file into DataFrame
attendance_df = pd.read_csv(attendanceFilePath)
attendance_df.head()

# Transform

### A. &nbsp;National Parks (from JSON)

In [ ]:
# Transform Location Number (park code) column to all caps
a_df["Location Number"] = a_df["Location Number"].str.upper()
a_df.head()

In [ ]:
# Drop unnecessary columns
a_df.drop(columns = ["Address", "City", "State", "Zip Code", "Phone Number", "Fax Number", "Location"], inplace = True)
a_df.head()

In [ ]:
# Rename columns
a_df = a_df.rename(columns = {"Location Number":"Code", "Location Name":"Name"})
a_df.head()

### B. &nbsp;National Parks (from CSV)

In [ ]:
# Create new DataFrame with only desired columns
b_df = bRaw_df[["UNIT_CODE", "UNIT_NAME"]].copy()
b_df.head()

In [ ]:
# Rename columns
b_df = b_df.rename(columns = {"UNIT_CODE":"Code", "UNIT_NAME":"Name"})
b_df.head()

In [ ]:
# Sort DataFrame by National Park name
b_df.sort_values(by = ["Name"], ascending = True, inplace = True)
b_df.head()

In [ ]:
# Reset index in place
b_df.reset_index(inplace = True, drop = True)
b_df.head()

##### *Merge A and B DataFrames to create one list of National Parks DataFrame*

In [ ]:
# Merge DataFrames
natParksRaw_df = pd.merge(b_df, a_df, how = "left", left_on = "Code", right_on = "Code")
natParksRaw_df

In [ ]:
# Drop unnecessary columns
natParksRaw_df.drop(columns = ["Name_y"], inplace = True)
natParksRaw_df.head()

In [ ]:
# Rename columns
natParksRaw_df = natParksRaw_df.rename(columns = {"Name_x":"Name"})
natParksRaw_df.head()

In [ ]:
# DataFrame size
natParksRaw_df.shape

In [ ]:
# Check number of unique values in Code column
a = len(pd.unique(natParksRaw_df["Code"]))
a

In [ ]:
# Identify rows with duplicate value in Code column
duplicates_df = natParksRaw_df[natParksRaw_df.duplicated("Code")]
duplicates_df

In [ ]:
# Drop rows with duplicates in Code column (acreage is totaled on first row of duplicates on natParksArea CSV)
natParksRaw_df = natParksRaw_df.drop_duplicates(subset = "Code", keep = "first")
natParksRaw_df

In [ ]:
# Create DataFrame from rows with missing lat/lng values
null_df = natParksRaw_df[natParksRaw_df.isnull().any(axis = 1)]
null_df

In [ ]:
# Check number of unique values in Code column
b = len(pd.unique(null_df["Code"]))
b

In [ ]:
# # Export DataFrame to CSV for manual lat/lng entry
# null_df.to_csv("data/missingCoordinates.csv")

In [ ]:
# Read CSV file into DataFrame
missingCoordinates_df = pd.read_csv("data/missingCoordinates.csv", encoding = "iso-8859-1")
missingCoordinates_df

In [ ]:
# Drop unwanted space from strings in Longitude column
missingCoordinates_df["Longitude"] = missingCoordinates_df["Longitude"].str.replace(" ", "")
missingCoordinates_df

In [ ]:
# Drop unnecessary column
missingCoordinates_df.drop(columns = ["Unnamed: 0"], inplace = True)
missingCoordinates_df.head()

In [ ]:
# Convert Longitude column to float
missingCoordinates_df["Longitude"] = missingCoordinates_df["Longitude"].astype(float)
missingCoordinates_df

In [ ]:
# Data types
missingCoordinates_df.dtypes

##### *Merge National Parks and Missing Coordinates DataFrames*

In [ ]:
# Merge DataFrames
np_df = natParksRaw_df.merge(missingCoordinates_df, how = "left", left_on = "Code", right_on = "Code")
np_df["Latitude_x"].fillna(np_df["Latitude_y"], inplace = True)
np_df["Longitude_x"].fillna(np_df["Longitude_y"], inplace = True)
np_df

In [ ]:
# Check number of unique values in Code column
c = len(pd.unique(np_df["Code"]))
c

In [ ]:
# Drop unnecessary columns
np_df.drop(columns = ["Name_y", "Latitude_y", "Longitude_y"], inplace = True)
np_df.head()

In [ ]:
# Rename columns
np_df = np_df.rename(columns = {"Name_x":"Name", "Latitude_x":"Latitude", "Longitude_x":"Longitude"})
np_df

In [ ]:
# DataFrame size
np_df.shape

### C. &nbsp;National Parks with Size in Acres (from CSV)

In [ ]:
# Create new DataFrame with only desired columns
c_df = cRaw_df[["UNIT_CODE", "UNIT_NAME", "Shape__Acreage"]].copy()
c_df.head()

In [ ]:
# Rename columns
c_df = c_df.rename(columns = {"UNIT_CODE":"Code", "UNIT_NAME":"Name", "Shape__Acreage":"Area"})
c_df.head()

In [ ]:
# Sort DataFrame by National Park name
c_df.sort_values(by = ["Name"], ascending = True, inplace = True)
c_df.head()

In [ ]:
# Reset index in place
c_df.reset_index(inplace = True, drop = True)
c_df

In [ ]:
# Check number of unique values in Code column
d = len(pd.unique(c_df["Code"]))
d

In [ ]:
# DataFrame size
c_df.shape

##### *Merge National Parks and National Parks Area DataFrames*

In [ ]:
# Merge DataFrames
natParksArea_df = np_df.merge(c_df, how = "left", left_on = "Code", right_on = "Code")
natParksArea_df

In [ ]:
# Check number of unique values in Code column
e = len(pd.unique(natParksArea_df["Code"]))
e

In [ ]:
# Drop unnecessary columns
natParksArea_df.drop(columns = ["Name_y"], inplace = True)
natParksArea_df.head()

In [ ]:
# Rename columns
natParksArea_df = natParksArea_df.rename(columns = {"Name_x":"Name"})
natParksArea_df

In [ ]:
# Reset index in place
natParksArea_df.reset_index(inplace = True, drop = True)
natParksArea_df

In [ ]:
# DataFrame size
natParksArea_df.shape

In [ ]:
# Create DataFrame to look for rows with NaN values
nullArea_df = natParksArea_df[natParksArea_df.isnull().any(axis = 1)]
nullArea_df

In [ ]:
# Append acreage area to ALKA
natParksArea_df["Area"][5] = 424.242424

### D. &nbsp;Points of Interest (from CSV)

In [ ]:
# Create new DataFrame with only desired columns
allPoints_df = rawPoints_df[["MAPLABEL", "POITYPE", "Y", "X"]].copy()
allPoints_df

In [ ]:
# Rename columns
allPoints_df = allPoints_df.rename(columns = {"MAPLABEL":"Name", "POITYPE":"Type", "Y":"Latitude", "X":"Longitude"})
allPoints_df.head()

In [ ]:
# Transform blank string values to NaN values
allPoints_df["Name"].replace(" ", np.nan, inplace = True)
allPoints_df

In [ ]:
# Drop rows containing NaN values
allPoints_df.dropna(subset = ["Name"], inplace = True)
allPoints_df

In [ ]:
# Reset index in place
allPoints_df.reset_index(inplace = True, drop = True)
allPoints_df.head()

In [ ]:
# # List all points of interest types
# allPoints_df["Type"].unique()

In [ ]:
# Data types
allPoints_df.dtypes

##### Parking Lots DataFrame

In [ ]:
# Declare boolean variable to filter by parking lots
parkingLots = allPoints_df["Type"] == "Parking Lot"

In [ ]:
# Create new DataFrame filtered on parking lots
parking_df = allPoints_df[parkingLots]
parking_df

In [ ]:
# Rename columns for SQL compatibility
parking_df = parking_df.rename(columns = {"Name":"name", "Type":"type", "Latitude":"latitude", "Longitude":"longitude"})
parking_df.head()

In [ ]:
# Reset index in place
parking_df.reset_index(inplace = True, drop = True)
parking_df.head()

##### Filtered Points of Interest for Map Markers

In [ ]:
# Create new DataFrame filtered on desired type values
filteredPoints_df = allPoints_df[(allPoints_df.Type == "ATM") |
                                 (allPoints_df.Type == "Entrance / Exit") |
                                 (allPoints_df.Type == "Fire Station") |
                                 (allPoints_df.Type == "First Aid Station") |
                                 (allPoints_df.Type == "Food Concession") |
                                 (allPoints_df.Type == "Food Service") |
                                 (allPoints_df.Type == "Gift Shop") |
                                 (allPoints_df.Type == "InfoKiosk") |
                                 (allPoints_df.Type == "Information") |
                                 (allPoints_df.Type == "Information Board") |
                                 (allPoints_df.Type == "Information Map") |
                                 (allPoints_df.Type == "Police") |
                                 (allPoints_df.Type == "Ranger Station") |
                                 (allPoints_df.Type == "Shelter") |
                                 (allPoints_df.Type == "Store") |
                                 (allPoints_df.Type == "Visitor Center") |
                                 (allPoints_df.Type == "Weather Shelter")]
filteredPoints_df

In [ ]:
# Rename columns for SQL compatibility
filteredPoints_df = filteredPoints_df.rename(columns = {"Name":"name", "Type":"type", "Latitude":"latitude",
                                                        "Longitude":"longitude"})
filteredPoints_df.head()

In [ ]:
# Reset index in place
filteredPoints_df.reset_index(inplace = True, drop = True)
filteredPoints_df.head()

In [ ]:
# Data types
filteredPoints_df.dtypes

### E. &nbsp;Attendance/Visitors (Recreation Visits; from CSV)
- By Park by Year
- "Presidents' Park" did not have its own code; combined these attendance values with White House attendance numbers.
- "Klondike Gold Rush - Seattle" technically falls under Klondike Gold Rush - Alaska and does not have its own code; eliminated this record altogether (average annual attendance is 66,907).

In [ ]:
# Rename column
attendance_df = attendance_df.rename(columns = {"Park Name":"Name"})
attendance_df.head()

In [ ]:
# Check number of unique values in Code column
f = len(pd.unique(attendance_df["Code"]))
f

In [ ]:
# Data types
attendance_df.dtypes

In [ ]:
# Transform blank string values to NaN values
attendance_df["2011"].replace(" ", np.nan, inplace = True)
attendance_df["2012"].replace(" ", np.nan, inplace = True)
attendance_df["2013"].replace(" ", np.nan, inplace = True)
attendance_df["2014"].replace(" ", np.nan, inplace = True)
attendance_df["2015"].replace(" ", np.nan, inplace = True)
attendance_df["2016"].replace(" ", np.nan, inplace = True)
attendance_df["2017"].replace(" ", np.nan, inplace = True)
attendance_df["2018"].replace(" ", np.nan, inplace = True)
attendance_df["2019"].replace(" ", np.nan, inplace = True)
attendance_df["2020"].replace(" ", np.nan, inplace = True)
attendance_df["Average"].replace(" ", np.nan, inplace = True)
attendance_df.head()

In [ ]:
# Remove commas from string values in yearly attendance columns
attendance_df["2011"] = attendance_df["2011"].str.replace(",", "")
attendance_df["2012"] = attendance_df["2012"].str.replace(",", "")
attendance_df["2013"] = attendance_df["2013"].str.replace(",", "")
attendance_df["2014"] = attendance_df["2014"].str.replace(",", "")
attendance_df["2015"] = attendance_df["2015"].str.replace(",", "")
attendance_df["2016"] = attendance_df["2016"].str.replace(",", "")
attendance_df["2017"] = attendance_df["2017"].str.replace(",", "")
attendance_df["2018"] = attendance_df["2018"].str.replace(",", "")
attendance_df["2019"] = attendance_df["2019"].str.replace(",", "")
attendance_df["2020"] = attendance_df["2020"].str.replace(",", "")
attendance_df["Average"] = attendance_df["Average"].str.replace(",", "")
attendance_df.head()

In [ ]:
# Transform NaN values to "0"
attendance_df["2011"] = attendance_df["2011"].fillna(0)
attendance_df["2012"] = attendance_df["2012"].fillna(0)
attendance_df["2013"] = attendance_df["2013"].fillna(0)
attendance_df["2014"] = attendance_df["2014"].fillna(0)
attendance_df["2015"] = attendance_df["2015"].fillna(0)
attendance_df["2016"] = attendance_df["2016"].fillna(0)
attendance_df["2017"] = attendance_df["2017"].fillna(0)
attendance_df["2018"] = attendance_df["2018"].fillna(0)
attendance_df["2019"] = attendance_df["2019"].fillna(0)
attendance_df["2020"] = attendance_df["2020"].fillna(0)
attendance_df["Average"] = attendance_df["Average"].fillna(0)
attendance_df.head()

In [ ]:
# Transform values to integers
attendance_df["2011"] = attendance_df["2011"].astype(int)
attendance_df["2012"] = attendance_df["2012"].astype(int)
attendance_df["2013"] = attendance_df["2013"].astype(int)
attendance_df["2014"] = attendance_df["2014"].astype(int)
attendance_df["2015"] = attendance_df["2015"].astype(int)
attendance_df["2016"] = attendance_df["2016"].astype(int)
attendance_df["2017"] = attendance_df["2017"].astype(int)
attendance_df["2018"] = attendance_df["2018"].astype(int)
attendance_df["2019"] = attendance_df["2019"].astype(int)
attendance_df["2020"] = attendance_df["2020"].astype(int)
attendance_df["Average"] = attendance_df["Average"].astype(int)
attendance_df.head()

In [ ]:
# Data types
attendance_df.dtypes

In [ ]:
# DataFrame size
attendance_df.shape

##### *Merge National Parks and Attendance DataFrames and create one comprehensive natParks DataFrame*

In [ ]:
# Merge DataFrames
natParks_df = natParksArea_df.merge(attendance_df, how = "left", left_on = "Code", right_on = "Code")
natParks_df

In [ ]:
# Drop unnecessary columns
natParks_df.drop(columns = ["Name_y"], inplace = True)
natParks_df.head()

In [ ]:
# Rename columns for compatibility with SQL
natParks_df = natParks_df.rename(columns = {"Code":"code", "Name_x":"name", "Latitude":"latitude", "Longitude":"longitude",
                                            "Area":"acres", "2011":"att_2011", "2012":"att_2012", "2013":"att_2013",
                                            "2014":"att_2014", "2015":"att_2015", "2016":"att_2016", "2017":"att_2017",
                                            "2018":"att_2018", "2019":"att_2019", "2020":"att_2020", "Average":"att_average"})
natParks_df.head()

In [ ]:
# Data types
natParks_df.dtypes

In [ ]:
# Transform NaN values to "0"
natParks_df["att_2011"] = natParks_df["att_2011"].fillna(0)
natParks_df["att_2012"] = natParks_df["att_2012"].fillna(0)
natParks_df["att_2013"] = natParks_df["att_2013"].fillna(0)
natParks_df["att_2014"] = natParks_df["att_2014"].fillna(0)
natParks_df["att_2015"] = natParks_df["att_2015"].fillna(0)
natParks_df["att_2016"] = natParks_df["att_2016"].fillna(0)
natParks_df["att_2017"] = natParks_df["att_2017"].fillna(0)
natParks_df["att_2018"] = natParks_df["att_2018"].fillna(0)
natParks_df["att_2019"] = natParks_df["att_2019"].fillna(0)
natParks_df["att_2020"] = natParks_df["att_2020"].fillna(0)
natParks_df["att_average"] = natParks_df["att_average"].fillna(0)
natParks_df.head()

In [ ]:
# Transform values to integers
natParks_df["att_2011"] = natParks_df["att_2011"].astype(int)
natParks_df["att_2012"] = natParks_df["att_2012"].astype(int)
natParks_df["att_2013"] = natParks_df["att_2013"].astype(int)
natParks_df["att_2014"] = natParks_df["att_2014"].astype(int)
natParks_df["att_2015"] = natParks_df["att_2015"].astype(int)
natParks_df["att_2016"] = natParks_df["att_2016"].astype(int)
natParks_df["att_2017"] = natParks_df["att_2017"].astype(int)
natParks_df["att_2018"] = natParks_df["att_2018"].astype(int)
natParks_df["att_2019"] = natParks_df["att_2019"].astype(int)
natParks_df["att_2020"] = natParks_df["att_2020"].astype(int)
natParks_df["att_average"] = natParks_df["att_average"].astype(int)
natParks_df

In [ ]:
# Data types
natParks_df.dtypes

# Load

In [ ]:
# Export DataFrames to CSV files
natParks_df.to_csv("resources/natParksFinal.csv")
filteredPoints_df.to_csv("resources/pointsFinal.csv")
parking_df.to_csv("resources/parkingFinal.csv")

In [ ]:
# # Export DataFrames to JSON files
# natParks_df.to_json("resources/natParksFinal.json")
# filteredPoints_df.to_json("resources/pointsFinal.json")
# parking_df.to_json("resources/parkingFinal.json")

##### postgreSQL Method (functioning correctly)

In [ ]:
# Create an engine that can talk to the database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
connection = engine.connect()

In [ ]:
# Check for table
engine.table_names()

In [ ]:
# Use pandas to load DataFrames into natParks table
natParks_df.to_sql(name = "natparks", con = engine, if_exists = "append", index = False)
filteredPoints_df.to_sql(name = "pointsinterest", con = engine, if_exists = "append", index = False)
parking_df.to_sql(name = "parkinglots", con = engine, if_exists = "append", index = False)

# Code below to be used for creating SQLite Database

##### Method 1. &nbsp;Insert Data with Classes and Constructors (10.2 Activities 1, 3, and 4)

In [ ]:
# # Sets an object to utilize the default declarative base in SQL Alchemy
# Base = declarative_base()

In [ ]:
# # Create classes to define our tables
# class Parks(Base):
#     __tablename__ = "natparks"
#     id = Column(Integer, primary_key = True)
#     code = Column(String(255))
#     name = Column(String(255))
#     latitude = Column(Float)
#     longitude = Column(Float)
#     acres = Column(Float)
#     att_2011 = Column(Integer)
#     att_2012 = Column(Integer)
#     att_2013 = Column(Integer)
#     att_2014 = Column(Integer)
#     att_2015 = Column(Integer)
#     att_2016 = Column(Integer)
#     att_2017 = Column(Integer)
#     att_2018 = Column(Integer)
#     att_2019 = Column(Integer)
#     att_2020 = Column(Integer)
#     att_average = Column(Integer)

# class Points(Base):
#     __tablename__ = "pointsinterest"
#     id = Column(Integer, primary_key = True)
#     name = Column(String(255))
#     type = Column(String(255))
#     latitude = Column(Float)
#     longitude = Column(Float)
    
# class Parking(Base):
#     __tablename__ = "parkinglots"
#     id = Column(Integer, primary_key = True)
#     name = Column(String(255))
#     type = Column(String(255))
#     latitude = Column(Float)
#     longitude = Column(Float)

##### NEED TO FIRST CREATE NATPARKS.SQLITE REFERENCED IN CELL BELOW???

In [ ]:
# # Create engine to natparks.sqlite
# engine = create_engine("sqlite:///resources/natparks.sqlite")

In [ ]:
# # Create our tables in the database
# Base.metadata.create_all(engine)

In [ ]:
# # Create our session (link) from Python to the DB
# session = Session(bind = engine)

In [ ]:
# # Create instances of each class
# parks = Parks("WHAT GOES IN HERE?")  "CONVERT DATAFRAMES TO DICTIONARY TO PASS IN HERE?"
# points = Points("WHAT GOES IN HERE?")
# lots = Parking("WHAT GOES IN HERE?")

In [ ]:
# # Use the SQL Alchemy methods to run simple "INSERT" statements using the classes and objects
# session.add(Parks)
# session.add(Points)
# session.add(Parking)

In [ ]:
# # Commit transaction of changes to database
# session.commit()

##### Method 2. &nbsp;Insert Data with Pandas and SQLAlchemy ORM

In [ ]:
# # Insert Data with Pandas and SQLAlchemy ORM
# Session = sessionmaker(bind = dest_db_con)
# session = Session()
# session.bulk_insert_mappings(natParks_df, df.to_dict(orient = "records"))
# session.close()